# 信贷风险分析

*Copyright (c) 2023 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

风险分析的任务是评估给定资产在面临金融系统波动时的风险，即在险价值 Value at Risk（VaR）。该价值可以衡量在金融系统下给定资产的潜在损失。决定资产在险价值的因素有很多，包括金融机构对于风险的不同偏好、金融系统的波动特征和资产本身的含风险性质。2014 年 Rutkowski 与 Tarca [1] 给出了一个计算风险资本的数学模型。该模型认为整体金融系统的波动应该遵循高斯布朗运动，从而能够计算出较高可信度下处于该系统中资产损失上界的最小值。

当应用于信贷风险分析问题时，该模型可以用于评估银行的信贷风险监管资本，即银行所持有信贷资产组合在当前金融系统下的在险价值。在该模型中，我们记给定的信贷资产组合为 $\{ 0, ..., K-1 \}$, 且决定资产组合总损失的随机变量为
$$
\mathcal{L} = \sum_{k=0}^{K - 1} \lambda_k X_k(Z)
。$$
这里，$\lambda_k$ 为第 $k$ 个信贷资产的违约损失（loss given default）；$Z$ 为描述金融系统波动过程的潜在随机变量（latent random variable），且其概率分布默认为标准正态分布；$X_k(Z)$ 为描述金融系统下资产违约过程的伯努利随机变量。具体地，当 $Z = z$ 时，随机变量 $X_k(Z)$ 的参数 $p_k(z)$，即第 $k$ 个资产的违约概率，取决于该资产对于系统风险的敏感度（sensitivity）$\rho_k \in [0, 1]$ 以及无系统风险下该资产的基础违约概率（base default probability）$p_k^{(0)} \in [0, 1]$。在标定置信度（confidence level）之后，信贷资产组合的在险价值应定义为『于置信度 $\alpha$ 下，信贷资产损失上界的最小值』，即
$$
\textrm{VaR}_\alpha(\mathcal{L}) := \inf \{ x \,|\, \textrm{Pr}(\mathcal{L} \leq x) \geq \alpha \}
。$$
例如，若银行持有的一组信贷资产在置信度 $99\%$ 下的在险价值正好为 $100$ 万，那么当该资产组合出现违约情况时，由于违约而造成的资产损失超过 $100$ 万的概率不高于 $1\%$。在经典计算中，信贷风险分析问题中的在险价值可以通过经典蒙特卡罗（classical Monte Carlo）方法和二分搜索（bisection search）方法估算：
1. 根据资产组合性质，选定初始 VaR 猜测值 $\check{x}$。
2. 使用经典蒙特卡罗算法估算概率值 $p_{\check{x}} = \textrm{Pr}(\mathcal{L} \leq \check{x})$。
3. 将概率值 $p_{\check{x}}$ 与 $\alpha$ 比较，并根据比较结果和二分搜索方法更新猜测值 $\check{x}$。
4. 若更新值达到收敛标准，输出 $\check{x} = \textrm{VaR}_\alpha(\mathcal{L})$；否则返回第 $2$ 步。

## 量子解决方案

不同于经典算法，量子计算可以使用量子振幅估计（quantum amplitude estimation）算法替换上述第 2 步中使用的经典蒙特卡罗算法，来提高概率值 $p_{\check{x}}$ 的估算效率。通过量子叠加和纠缠的特性，量子方案与经典方案相比有望在采样次数上获取平方加速的优势 [2]。接下来我们展示如何使用量桨来模拟该量子方案，从而完成信贷风险分析的在险价值计算问题。

### 在线演示

我们已经给出了一个设置好的参数，可以直接用于风险资产组合的在险价值计算。只需要在 `config.toml` 这个配置文件中进行对应的配置，然后输入命令 
`python credit_risk.py --config config.toml`
即可对配置好的资产组合进行在险价值计算。

这里，我们给出一个在线演示的版本，可以在线进行测试。首先定义配置文件的内容：

In [1]:
cra_toml = r"""
# 用于信贷风险分析模型的整体配置文件。
# 资产数量。
num_assets = 4
# 基础违约概率。
base_default_prob = [0.15, 0.25, 0.39, 0.58]
# 敏感度。
sensitivity = [0.37, 0.21, 0.32, 0.02]
# 违约损失。
lgd = [5, 1, 3, 4]
# 置信度。
confidence_level = 0.99
# 估计精度系数。系数越高，则估计的结果越精确。
degree_of_simulation = 4
"""

量桨的金融模块实现了量子解决方案的数值模拟。我们可以从 ``paddle_quantum.finance`` 模块里导入 ``CreditRiskAnalyzer`` 来解决配置好的在险价值计算问题。

In [2]:
import os
import warnings
warnings.filterwarnings("ignore")
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

import toml
from paddle_quantum.finance import CreditRiskAnalyzer

config = toml.loads(cra_toml)
num_assets = config["num_assets"]
base_default_prob = config["base_default_prob"]
sensitivity = config["sensitivity"]
lgd = config["lgd"]
confidence_level = config["confidence_level"]
degree_of_simulation = config["degree_of_simulation"]

estimator = CreditRiskAnalyzer(num_assets, base_default_prob, sensitivity, lgd, confidence_level, degree_of_simulation)
print("该资产组合的在险价值大约为", estimator.estimate_var())

-----------------------------------------------------------------------------------
Begin bisection search for VaR with confidence level >= 99.0%.
-----------------------------------------------------------------------------------
Lower guess: level            Middle guess: level            Upper guess: level    
    -1     : 0.000                  6    : 0.691                 13     : 1.000    
     6     : 0.691                  9    : 0.941                 13     : 1.000    
     9     : 0.941                 11    : 0.962                 13     : 1.000    
    11     : 0.962                 12    : 0.990                 13     : 1.000    
-----------------------------------------------------------------------------------
Estimated VaR is 12 with confidence level 99.0%.
-----------------------------------------------------------------------------------
该资产组合的在险价值大约为 12


___

# 注意事项

这里提供的模型只用于特定模型的信贷风险分析问题。

# 引用信息

```
@article{rutkowski2015regulatory,
  title={Regulatory capital modeling for credit risk},
  author={Rutkowski, Marek and Tarca, Silvio},
  journal={International Journal of Theoretical and Applied Finance},
  volume={18},
  number={05},
  pages={1550034},
  year={2015},
  publisher={World Scientific}
}

@article{egger2020credit,
  title={Credit risk analysis using quantum computers},
  author={Egger, Daniel J and Guti{\'e}rrez, Ricardo Garc{\'\i}a and Mestre, Jordi Cahu{\'e} and Woerner, Stefan},
  journal={IEEE Transactions on Computers},
  volume={70},
  number={12},
  pages={2136--2145},
  year={2020},
  publisher={IEEE}
}
```

# 参考文献

[1] Rutkowski, Marek, and Silvio Tarca. "Regulatory capital modeling for credit risk." International Journal of Theoretical and Applied Finance 18.05 (2015): 1550034.

[2] Egger, Daniel J., et al. "Credit risk analysis using quantum computers." IEEE Transactions on Computers 70.12 (2020): 2136-2145.